# 06 Custom Configuration (Experiment)
Read first: `examples/02_custom_configuration.py`

Explore composition knobs: add tools and/or switch to streaming orchestrator.


## Select a provider
Set `PROVIDER_KEY` to `sonnet`, `opus`, `gpt`, or `gpt-codex` and run the next cell.
The helper will set env vars (prompting if missing) and print the selection.


In [29]:
# Choose your provider key (sonnet | opus | gpt | gpt-codex)
PROVIDER_KEY = "gpt"

In [30]:
# Resolve foundation/provider paths and ensure env var
import importlib.util
import os
from pathlib import Path


def find_repo_root() -> Path:
    candidates = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
    for p in candidates:
        if (p / "utils" / "providers.py").exists():
            return p
    return Path.cwd()


ROOT = find_repo_root()
providers_path = ROOT / "utils" / "providers.py"
spec = importlib.util.spec_from_file_location("providers_helper", providers_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)
select_provider = module.select_provider
print_provider_menu = module.print_provider_menu

FOUNDATION_PATH, PROVIDER_PATH, REQUIRED_ENV = select_provider(PROVIDER_KEY, foundation="minimal")
print_provider_menu(PROVIDER_KEY)
print(f"Selected: {PROVIDER_KEY} -> {PROVIDER_PATH.name}")
env_set = "Yes" if os.environ.get(REQUIRED_ENV) else "No"
print(f"Required env: {REQUIRED_ENV} (set? {env_set})")

Providers (set PROVIDER_KEY):
     sonnet: anthropic-sonnet.yaml
     opus: anthropic-opus.yaml
  -> gpt: openai-gpt.yaml
     gpt-codex: openai-gpt-codex.yaml
Selected: gpt -> openai-gpt.yaml
Required env: OPENAI_API_KEY (set? Yes)


In [31]:
# Optional: set API key here (safer: export in terminal)
import getpass
import os

if not os.getenv(REQUIRED_ENV):
    os.environ[REQUIRED_ENV] = getpass.getpass(f"Enter {REQUIRED_ENV}: ")
    print(f"Set {REQUIRED_ENV} for this kernel session.")
else:
    print(f"{REQUIRED_ENV} already set.")

OPENAI_API_KEY already set.


Configure overlays below. Turn them on/off and re-run the next cell.
- Tools overlay adds filesystem + bash tools (downloads modules on first run).
- Streaming overlay swaps to loop-streaming and streaming UI hook.


In [32]:
# Toggles and parameters (edit me)
USE_TOOLS = True
USE_STREAMING = False
PROMPT = "List the files in the current directory and tell me what you find."

In [33]:
from amplifier_foundation import Bundle
from amplifier_foundation import load_bundle


def overlays():
    opts = []
    if USE_TOOLS:
        opts.append(
            Bundle(
                name="tools-config",
                version="1.0.0",
                tools=[
                    {
                        "module": "tool-filesystem",
                        "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main",
                    },
                    {
                        "module": "tool-bash",
                        "source": "git+https://github.com/microsoft/amplifier-module-tool-bash@main",
                    },
                ],
            )
        )
    if USE_STREAMING:
        opts.append(
            Bundle(
                name="streaming-config",
                version="1.0.0",
                session={
                    "orchestrator": {
                        "module": "loop-streaming",
                        "source": "git+https://github.com/microsoft/amplifier-module-loop-streaming@main",
                    }
                },
                hooks=[
                    {
                        "module": "hooks-streaming-ui",
                        "source": "git+https://github.com/microsoft/amplifier-module-hooks-streaming-ui@main",
                    }
                ],
            )
        )
    return opts


async def run(prompt: str):
    foundation = await load_bundle(str(FOUNDATION_PATH))
    provider = await load_bundle(str(PROVIDER_PATH))
    composed = foundation.compose(provider)
    for overlay in overlays():
        composed = composed.compose(overlay)
    prepared = await composed.prepare()
    session = await prepared.create_session()
    async with session:
        return await session.execute(prompt)


response = await run(PROMPT)
print(response)

No approval system provided - approval requests will fail
No display system provided - hook messages will be logged only


Here’s what’s in the current directory (from the listing you provided):

- `README.md` (2.4 KB)
- 13 Jupyter notebooks, numbered as a sequence of examples/tutorials:
  - `01_hello_world.ipynb`
  - `02_custom_configuration.ipynb`
  - `03_custom_tool.ipynb`
  - `04_load_and_inspect.ipynb`
  - `05_composition.ipynb`
  - `06_validation_checker.ipynb`
  - `07_bundle_playground.ipynb`
  - `08_full_workflow.ipynb`
  - `09_multi_agent_system.ipynb`
  - `10_provider_comparison.ipynb`
  - `11_approval_gates.ipynb`
  - `12_session_persistence.ipynb`
  - `13_composition_explorer.ipynb`

What I find: this directory looks like a small workshop/course repo centered around notebooks that build from “hello world” through custom tools/configuration, composition, validation, multi-agent workflows, provider comparisons, approval gates, and session persistence, with a `README.md` likely explaining how to run/follow along.


### Try this
- Toggle `USE_TOOLS`/`USE_STREAMING` and rerun.
- Change tool list to your own modules (add/remove dict entries).
- Swap `PROVIDER_PATH` to compare models.
